1. Establish the connection with google cloud sql

In [1]:
#!pip install cloud-sql-python-connector   
# Firstly, installing this package to connect cloud sql!!!

In [2]:
from google.cloud.sql.connector import Connector, IPTypes
import sqlalchemy
from google.oauth2 import service_account
from sqlalchemy import text
import pandas as pd

In [3]:
# Specify the service account file path
SERVICE_ACCOUNT_FILE = r'C:\Users\User\Desktop\read_me\qf5214_professor.json'
                                                          
# Create service account credentials
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE
)
connector = Connector(credentials=credentials)

In [4]:
def getconn() -> sqlalchemy.engine.base.Connection:
    instance_connection_string = "supple-folder-418707:us-central1:qf5214project-replica"
    #instance_connection_string = "supple-folder-418707:us-central1:qf5214project"
    db_user = "user_professor"
    db_pass = "qf5214_group4"
    db_name = "Bitcoin_data"                                       # Replace with your actual database name, but none seems to work

    conn = connector.connect(
        instance_connection_string,
        "pymysql",  
        user=db_user,
        password=db_pass,
        db=db_name,
        ip_type=IPTypes.PUBLIC,  # Use public IP for access
    )
    return conn

# Create the SQLAlchemy engine
engine = sqlalchemy.create_engine(
    "mysql+pymysql://",  # Our database is MySQL8.0
    creator=getconn,
)

2. Get the name of table in database

In [5]:
with engine.connect() as conn:
    # Perform a query to get all table names in the bitcoin database
    result = conn.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema = 'Bitcoin_data';"))
    # Print out the name of each form
    for row in result:
        print(row[0])  

market
news
talk_comments
talk_subjects
talk_users


3. Get the table in database in form of dataframe

In [6]:
query = "SELECT * FROM news"
news_data = pd.read_sql(query, engine)
print('The shape of this table is ',news_data.shape)
news_data.head(5)

The shape of this table is  (183879, 10)


,news_id,date,news_title,news_url,news_domain,news_language,news_country,news_title_emotion,news_title_subjectivity,news_important
0,0,2017-01-01,Bitcoin Price : A Breakthrough Moment for Bloc...,http://news.carbon-future.co.uk/archives/14260,news.carbon-future.co.uk,English,United Kingdom,0.000000,0.000000,0
1,1,2017-01-01,The Microsoft Office 365 to Include Bitcoin Su...,http://www.newsbtc.com/2017/01/01/microsoft-of...,newsbtc.com,English,United States,0.000000,0.000000,0
2,2,2017-01-01,Opinion : What Do We Gain by Sending Bitcoin t...,http://www.newsbtc.com/2017/01/01/sending-bitc...,newsbtc.com,English,United States,0.000000,0.000000,0
3,3,2017-01-01,Bitcoin Price Gains Over 50 Percent in 2016,http://www.newsbtc.com/2017/01/01/2016-good-ye...,newsbtc.com,English,United States,0.000000,0.000000,0
4,4,2017-01-01,Microsoft adds advanced support for bitcoin to...,http://bravenewcoin.com/news/microsoft-adds-ad...,bravenewcoin.com,English,United States,0.445455,0.684849,0


In [7]:
query = "SELECT * FROM market"
market_data = pd.read_sql(query, engine)
print('The shape of this table is ',market_data.shape)
market_data.head(5)       
#some data may not exist at beginning 

The shape of this table is  (5031, 11)


,date,btc_price,btc_volume,eth_price,eth_volume,cpi,nonfarm,population,nofarm_ratio,fed_rate,sofr
0,2008-01-01,NaN,NaN,NaN,NaN,211.080,138397.0,303280.0,0.4563,3.94,0.0
1,2008-02-01,NaN,NaN,NaN,NaN,211.693,138326.0,303494.0,0.4558,2.98,0.0
2,2008-03-01,NaN,NaN,NaN,NaN,213.528,138256.0,303707.0,0.4552,2.61,0.0
3,2008-04-01,NaN,NaN,NaN,NaN,214.823,138037.0,303926.0,0.4542,2.28,0.0
4,2008-05-01,NaN,NaN,NaN,NaN,216.632,137847.0,304157.0,0.4532,1.98,0.0


In [8]:
query = "SELECT * FROM talk_subjects"
talk_subjects = pd.read_sql(query, engine)
print('The shape of this table is ',talk_subjects.shape)
talk_subjects.head(5)

The shape of this table is  (60028, 6)


,talk_subject,date,talk_subject_replies,talk_subject_views,talk_subject_emotion,talk_subject_subjectivity
0,==- Sim Kids - What happe...,2020-06-23,25,292,-0.5,0.3
1,Bank of England Governor Claims Bitcoin...,2018-02-21,2,131,0.0,0.0
2,111111111111111113rZwuYDQxa1 can you get...,2016-10-31,31,1426,0.5,0.5
3,Bitcoin can replace the credit card today,2014-03-10,74,5090,0.0,0.0
4,Bitcoin Mentioned on Alex Jones,2012-06-17,5,1433,0.0,0.0


In [9]:
query = "SELECT * FROM talk_users"
talk_users = pd.read_sql(query, engine)
print('The shape of this table is ',talk_users.shape)
talk_users.head(5)

The shape of this table is  (90786, 3)


,talk_user,talk_user_activity,talk_user_merit
0,!!!Shazam!!!,14,0
1,!saac,10,0
2,#bitcoin,13,0
3,#BitcoinVegan,76,10
4,#BiT_pOL,133,100


In [10]:
# Note: The table is too big and it takes a long time to output
query = "SELECT * FROM talk_comments"
talk_comments = pd.read_sql(query, engine)
print('The shape of this table is ',talk_comments.shape)   
talk_comments.head(5)

The shape of this table is  (2415194, 6)


,talk_comment_id,date,talk_subject,talk_user,talk_comment,talk_comment_sentiment
0,0,2024-04-01,Can or should you close and reopen binance acc...,pornluver,is it possible to reopen binance account after...,-0.3818
1,1,2024-03-29,How would you dispose of a steel plate seed ba...,hatshepsut93,this thread sparked a question in my head. ima...,-0.3736
2,2,2024-03-29,How would you dispose of a steel plate seed ba...,Zaguru12,the simplest option is transferring the coins ...,-0.4404
3,3,2024-03-29,How would you dispose of a steel plate seed ba...,Smartvirus,"quote from: zaguru12 on march 29, 2024, 11:44:...",0.0310
4,4,2024-03-29,How would you dispose of a steel plate seed ba...,hatshepsut93,"quote from: zaguru12 on march 29, 2024, 11:44:...",-0.4588
